In [1]:
# Common 
import tensorflow
import numpy as np

# Model Definition 
import os
from keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

# Training
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from time import strftime

# Visualizations 
from keras.utils import plot_model

# Image pre-processing
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
image_size = 224
color_mode = 'rgb'     # "grayscale" or "rgb"

### Get Train and Validation Data

In [3]:
def getTrainAndTestSet(image_size, batch_size, color_mode):
    train_data_dir = './datasets/equal_data/gender/train/'
    valid_data_dir = './datasets/equal_data/gender/valid/'
    
    train_datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        color_mode=color_mode,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    valid_datagen = ImageDataGenerator(rescale=1.0/255)
    valid_generator = valid_datagen.flow_from_directory(
        valid_data_dir,
        color_mode=color_mode,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    num_samples=train_generator.samples
    num_classes=train_generator.num_classes
    num_men = sum(train_generator.classes == 1)
    num_woman = sum(train_generator.classes ==0)
    print("num woman:", num_woman)
    print("num men:", num_men)
    num_valid = valid_generator.samples
    
    return train_generator, valid_generator, num_valid, num_samples

### Define Model

In [4]:
def getModel_vgg11(image_shape, num_classes, name="vgg11"):
    ## VGG16 architecture: https://arxiv.org/pdf/1409.1556.pdf
    ## Also may be able to use VGG-Face pre-trained weights. Not sure if the VGG-Face
    ## architecture is the same as the VGG-16. 
    ## Note: if the architectures don't match, they probably only differ by a small 
    ## amount, so we can probably create a separate VGG-Face model based on our VGG16
    ## and then use the weights from Oxford: http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ 
    
    # This implementation is based on Configuration D from page 3 of 1409.1556.pdf, so 16 weight layers total: 
    
    # Input (image): 
    # Note, I read somewhere that for tensorflow the order matters for performance, 
    # so check if this should be (1, img_size, img_size) instead? 
    image_input = Input(shape=image_shape, name="image_input")
    
    # Note about pre-trained weights: 
    # We have to do some potentially different pre-processing depending on which 
    # pre-trained weights we use (if we use any). For example per-pixel mean-centering: 
    # https://gist.github.com/ksimonyan/211839e770f7b538e2d8#file-readme-md 
    
    # Group 1: 
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g1_01")(image_input)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g1")(x)
    
    # Group 2: 
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g2_01")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g2")(x)
        
    # Group 3: 
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g3_01")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g3_02")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g3")(x)
        
    # Group 4: 
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g4_01")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g4_02")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g4")(x)
        
    # Group 5:
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g5_01")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g5_02")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g5")(x)
    
    # Final weight layers:
    # Note: The ReLu and Dropout stages here aren't part of original VGG paper, but the website by the authors
    # of the paper lists a slightly different version of the paper model that is their "best", which 
    # includes these layers (http://www.robots.ox.ac.uk/~vgg/research/very_deep/): 
    x = Flatten()(x)
    x = Dense(units=4096, name="final_fc_1")(x)
    x = Activation(activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=4096, name="final_fc_2")(x)
    x = Activation(activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(units=num_classes, activation='softmax', name='predictions')(x)
#     x = Dense(units=1, activation="sigmoid", name="predictions")(x)
    
    # VGG paper says "Finally, to obtain a fixed-size vector of class scores for the image, 
    # the class score map is spatially averaged (sum-pooled)."
    # Not sure if/where to add that in. 
    # UDPATE: We probably don't need it. It's not mentioned in their 
    # website when they define their "best" version of their vgg16 and vgg19 architectures. 
    # x = GlobalAveragePooling2D()(x)
    
    model = Model(inputs=image_input, outputs=x, name=name)

    return model


In [5]:
def getModel_vgg16(image_shape, num_classes, name="vgg16", weights=""):
    ## VGG16 architecture: https://arxiv.org/pdf/1409.1556.pdf
    ## Also may be able to use VGG-Face pre-trained weights. Not sure if the VGG-Face
    ## architecture is the same as the VGG-16. 
    ## Note: if the architectures don't match, they probably only differ by a small 
    ## amount, so we can probably create a separate VGG-Face model based on our VGG16
    ## and then use the weights from Oxford: http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ 
    
    # This implementation is based on Configuration D from page 3 of 1409.1556.pdf, so 16 weight layers total: 
    
    # Input (image): 
    # Note, I read somewhere that for tensorflow the order matters for performance, 
    # so check if this should be (1, img_size, img_size) instead? 
    image_input = Input(shape=image_shape, name="image_input")
    
    # Note about pre-trained weights: 
    # We have to do some potentially different pre-processing depending on which 
    # pre-trained weights we use (if we use any). For example per-pixel mean-centering: 
    # https://gist.github.com/ksimonyan/211839e770f7b538e2d8#file-readme-md 
    
    # Group 1: 
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g1_01")(image_input)
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g1_02")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g1")(x)
    
    # Group 2: 
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g2_01")(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g2_02")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g2")(x)
        
    # Group 3: 
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g3_01")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g3_02")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g3_03")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g3")(x)
        
    # Group 4: 
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g4_01")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g4_02")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g4_03")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g4")(x)
        
    # Group 5:
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g5_01")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g5_02")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu", name="conv2d_g5_03")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="maxpool_g5")(x)
    
    # Final weight layers:
    # Note: The ReLu and Dropout stages here aren't part of original VGG paper, but the website by the authors
    # of the paper lists a slightly different version of the paper model that is their "best", which 
    # includes these layers (http://www.robots.ox.ac.uk/~vgg/research/very_deep/): 
    x = Flatten()(x)
    x = Dense(units=4096, name="final_fc_1")(x)
    x = Activation(activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=4096, name="final_fc_2")(x)
    x = Activation(activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(units=num_classes, activation='softmax', name='predictions')(x)
#     x = Dense(units=1, activation="sigmoid", name="predictions")(x)
    
    # VGG paper says "Finally, to obtain a fixed-size vector of class scores for the image, 
    # the class score map is spatially averaged (sum-pooled)."
    # Not sure if/where to add that in. 
    # UDPATE: We probably don't need it. It's not mentioned in their 
    # website when they define their "best" version of their vgg16 and vgg19 architectures. 
    # x = GlobalAveragePooling2D()(x)
    
    model = Model(inputs=image_input, outputs=x, name=name)
    
    if weights != "":
        weights_dir = "./weights/"
        weights_path = os.path.join(weights_dir, weights)
        model.load_weights(weights_path, by_name=True)
    
    return model

In [7]:
model = getModel_vgg16(
    image_shape = (image_size, image_size, 3) if color_mode=="rgb" else (image_size, image_size, 1)
    , num_classes = 2
#     , weights = "vgg16"
)

# plot_model(model, show_shapes=True, show_layer_names=False)

optimizer = optimizers.SGD(lr=1e-2, decay=5e-4, momentum=0.9, nesterov=True)
model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

### Hyperparameters:


In [8]:
batch_size = 64
num_epochs = 74
model_name = "vgg16"


### Train Model: 

In [9]:
##
## Get Data batches: 
train_generator, valid_generator, num_valid, num_samples = getTrainAndTestSet(image_size, batch_size, color_mode)

Found 17512 images belonging to 2 classes.
Found 1944 images belonging to 2 classes.
num woman: 8756
num men: 8756


In [ ]:
##
## Train: 
checkpointer = ModelCheckpoint(
    filepath = './weights/{}-{}.hdf5'.format(
        model_name
        , strftime("%a_%d_%b_%Y_%H_%M_%S")
    )
    , verbose = 1
    , save_best_only = True
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.00000001, verbose=1)
callbacks = [reduce_lr, checkpointer]
# callbacks = [reduce_lr]

history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              validation_steps=num_valid//batch_size,
                              steps_per_epoch=num_samples//batch_size, 
                              epochs=num_epochs,
                              callbacks=callbacks,
                              verbose=1,
                              use_multiprocessing=True,
                              workers=4)

Epoch 1/74
273/273 [==============================] - 154s 563ms/step - loss: 0.6933 - acc: 0.5015 - val_loss: 0.6925 - val_acc: 0.5005

Epoch 00001: val_loss improved from inf to 0.69252, saving model to ./weights/vgg16-Mon_16_Apr_2018_18_53_00.hdf5
Epoch 2/74
273/273 [==============================] - 147s 540ms/step - loss: 0.6808 - acc: 0.5660 - val_loss: 0.6511 - val_acc: 0.6312

Epoch 00002: val_loss improved from 0.69252 to 0.65110, saving model to ./weights/vgg16-Mon_16_Apr_2018_18_53_00.hdf5
Epoch 3/74
273/273 [==============================] - 150s 548ms/step - loss: 0.6416 - acc: 0.6341 - val_loss: 0.6187 - val_acc: 0.6729

Epoch 00003: val_loss improved from 0.65110 to 0.61873, saving model to ./weights/vgg16-Mon_16_Apr_2018_18_53_00.hdf5
Epoch 4/74
162/273 [================>.............] - ETA: 57s - loss: 0.6155 - acc: 0.6643